# cjio API design


In [1]:
from datetime import datetime
i = datetime.now()
print("Version {}-{}-{}".format(i.year, i.month, i.day))

Version 2019-5-20


## Why?

Let's take a look at the problems that I'm trying to solve with the API:

1. *It is cumbersome to work with 3D city models in a Python script or program, partially because the data model is complex and libraries for parsing it don't really exist.* Why Python? Because in my opinion 3D city models should be similarly easy to work with as other data models/formats in the analysis pipeline. Python is one of the most common scripting languages for data analysis, which includes GIS. Also this is the language that we teach to our students at the Geomatics MSc.

  + Luckily we have CityJSON already, which simplifies the data model part considerably. Also cjio works well in the command line. But in my experience, and according to a few other people, it would be handy to have a python library that allows to easily operate on a 3D city model. I'm heavily influenced by the *tidyverse* ecosystem in R, and also by the *pandas*, *scikit-learn* libraries in python. Thus in my head, cjio would allow to work with data as easily as these packages do, and also integrate with them.
  
## What has been done already?

When I started, cjio already had a well developed CLI. The idea is to expose the same functionality through an API, and also to provide and object-based interface to CityJSON.

Also there is [citygson](https://github.com/citygml4j/citygson), a Gson based library for parsing and serializing CityJSON. This is a Java library.

## The API

I approach the API design from the workflow that I aim at. As in **data preparation**, which entails reading a city model into python in a way that it becomes easy to work with each CityObject individually. Then **feature generation**, which is computing any measures, statistics from the CityObjects which can be fed into an analysis process.

### 1. Data preparation


#### Some requirements, ideas
+ Get footprints, wall, roofs from LoD1 AND LoD2

+ How to work with a 3d model and its pointcloud?

#### CityObject types

A cityobject with children.


In [2]:
co_1 = {
"CityObjects": {
  "id-1": {
    "type": "Building",
    "geographicalExtent": [ 84710.1, 446846.0, -5.3, 84757.1, 446944.0, 40.9 ], 
    "attributes": { 
      "measuredHeight": 22.3,
      "roofType": "gable",
      "owner": "Elvis Presley"
    },
    "children": ["id-2"],
    "geometry": [{...}]
  },
  "id-2": {
    "type": "BuildingPart", 
    "parents": ["id-1"],
    "children": ["id-3"]
  },
  "id-3": {
    "type": "BuildingInstallation", 
    "parents": ["id-2"]
  },
  "id-4": {
    "type": "LandUse"
  }
}
}

Using a single getter function, and pass the type as argument. It should get both 1st-level and 2nd-level city objects. But in case of 2nd-level objects, how do we keep the reference to the parents?


In [ ]:
def get_cityobjects(type):
    """Return a generator over the CityObjects of the given type. Type can be 1st-level or 2nd-level CityObject."""
    if type is None:
        yield all cityobjects
    else:
        yield cityobjects of the given type

def get_children():
    if cityobject has children:
        yield list of children
    else:
        yield list()

def get_parents():
    if cityobject has parents:
        yield list of parents
    else:
        yield list()

cm = cjio.load("some_model.json")
buildings = cm.get_cityobjects("building")
for building in buildings:
    children = building.get_children()

buildingparts = cm.get_cityobjects("buildingpart")
for part in buildingparts:
    part.get_children()
    part.get_parents()

#### Working with semantics

The initial idea for getting semantic surfaces was like this below. But I think it makes a cleaner API if we have a semantic surface getter method and we pass the surface type as parameter.


In [ ]:
cm = cjio.load("some_model.json")
building_1 = cm.building.get(1)
roof_geom = building_1.roofsurface.geometry

Let's make some test data. The idea is to test how can we link the semantic object to the geometry boundaries and its parts. Below is a Geometry `g` with a boundary that is composed of a composite solid and a solid. Also, the Semantic Surface `sa` which relates to the surfaces of the solids.


In [5]:
# a composite solid and a solid
g = [
    [
        [
            [[0, 3, 2, 1, 22]], [[4, 5, 6, 7]], [[0, 1, 5, 4]], [[1, 2, 6, 5]]
        ],
        [
            [[240, 243, 124]], [[244, 246, 724]], [[34, 414, 45]], [[111, 246, 5]]
        ]
    ],
    [
        [[[666, 667, 668]], [[74, 75, 76]], [[880, 881, 885]], [[111, 122, 226]]]
    ]
]

# semantic surface object
sa = {
     "surfaces" : [
         {
             "type": "WallSurface",
             "slope": 33.4,
             "children": [2]
         },
         {
             "type": "RoofSurface",
             "slope": 66.6
         },
         {
             "type": "Door",
             "parent": 0,
             "colour": "blue"
         }
     ],
     "values": [
       [ [0, 1, 2, None], [0, 1, 2, None]],
       [ None ]
     ]
}


Here we have a `Boundary` class and a `Geometry` class. The main idea is that `SemanticSurface`-es do not get their own class, thus we don't create objects from them. All the semantic information is stored in the Geometry object and the surfaces with semantics are simply extracted on request with the `Geometry.get_surfaces()` method. Here we implement the exact same indexing mechanism as in the data model.

The `Boundary` class simply stores the geometry boundary as it is stored in CityJSON. In the future its possible to implement "real" geometric object from some library if we find one, however in python not many exists. It is still an open question whether we should dereference the boundary and store the geometries Simple Feature-style, *or* keep a vertex list and use indices. But if we use the SF-style geometries, it's much easier to operate on them for the user, since they are self-contained, no need to look for the coordinates.


In [2]:
from typing import List, Dict
from copy import deepcopy

class Boundary(object):
    """CityJSON Geometry boundary

    .. note:: For now it holds the boundary as it is in the CityJSON file.
    In the future this class could implement real 3D geometric
    objects.
    """

    def __init__(self, vertices: List=list(), geom: List=list()):
        self.vertices = vertices
        self.geometry = geom

    # If we use a 'property' instead of a plain attribute we
    # can implement validation in the setter/getter/deleter methods
    # Also, we can later change the logic of the geometry property
    # without changing the interface, thus no regression.
    @property
    def geometry(self):
        """A nested array of vertex indices"""
        return self.__geometry

    @geometry.setter
    def geometry(self, geom: List):
        # we could do validation here
        if not isinstance(geom, list):
            raise TypeError("geometry must be a list")
        else:
            self.__geometry = geom

    @geometry.deleter
    def geometry(self):
        self.__geometry = list()


class Geometry(object):
    """CityJSON Geometry object

    .. seealso:: :class:`Boundary`
    """

    def __init__(self, type: str=None, lod: int=None, boundaries: List=None, semantics_obj: Dict=None):
        self.type = type
        self.lod = lod
        self.boundaries = boundaries
        self.semantics_obj = semantics_obj

    @property
    def semantics(self):
        """The Semantic Surface types in the Geometry"""
        return (s['type'] for s in self.semantics_obj['surfaces'])

    def get_surfaces(self, type=None):
        """The surfaces in the boundaries that have semantics"""
        sfc = []
        if self.type != 'CompositeSolid':
            raise NotImplementedError("sorry, only CompositeSolid works")
        else:
            # TODO: check if faster with generators
            for i, solid in enumerate(self.semantics_obj['values']):
                if solid:
                    for j, shell in enumerate(solid):
                        if shell:
                            for k, surface in enumerate(shell):
                                if surface is not None:
                                    if type:
                                        if type.lower() == self.semantics_obj['surfaces'][surface]['type'].lower():
                                            sfc.append(self.boundaries.geometry[i][j][k])
                                    else:
                                        sfc.append(self.boundaries.geometry[i][j][k])
        return sfc

Since the geometry boundary is a property not a boundary, we can implement custom getters/setters/deleters, thus when we delete the boundary the property is set to an empty list instead of `None`. Its seems to be a just a niceity for now, might be more useful later.


In [6]:
b = Boundary()
b.geometry
b.geometry = deepcopy(g)
del b.geometry
print(type(b.geometry))

<class 'list'>


In [7]:
comp_solid = Geometry(type="CompositeSolid",
                      lod=2,
                      boundaries=Boundary(geom=deepcopy(g)),
                      semantics_obj=deepcopy(sa)
                      )

# test semantics property
print(list(comp_solid.semantics) == ['WallSurface', 'RoofSurface', 'Door'])
# test get_surfaces method
print(comp_solid.get_surfaces('roofsurface') == [ [[4, 5, 6, 7]],[[244, 246, 724]] ])

True
True


Then this is more or less how the chain of working with semantic surfaces would look like.


In [ ]:
cm = cjio.load("some_model.json")
for building in cm.get_cityobjects("building"):
    geometry = building.geometry
    isinstance(geometry, Geometry)
    geometry.lod
    geometry.type
    # return the complete boundary of a single CityObject
    geometry.boundaries
    # return all the vertices of the boundary
    geometry.boundaries.vertices

Okay, but if we don't have a SemanticSuface class, it won't be possible to get the children and parents of a particular Semantic Surface like below.


In [ ]:
roofs = building.get_surfaces('roofsurface')
walls = building.get_surfaces('wallsurface')
grounds = building.get_surfaces('groundsurface')
for roof in roofs:
    geometry = roof.geometry
    geometry.boundaries
    roof.type
    roof.attributes
    roof.children
    roof.parents

**Vertex list for Boundary definition**

One alternative for storing boundaries would be to keep a "central" vertex list and the Boundary objects would only store the pointers to the vertices in the list. Exactly as it is defined in the CityJSON data model. The way I know how to solve this is to store the vertex list in a CityModel class, and the Geometry/Boundary classes would its subclass. This way a Boundary instance can access the CityModel vertices attribute.

Something like this below. If we have the vertices as a property or not, doesn't matter much.


In [ ]:
class CityModel(object):
    """Equivalent to the main CityJSON object in the data model"""
    type = 'CityJSON'
    cityjson_version = '1.0'

    def __init__(self, cityobjects, vtx):
        self.cityobjects = cityobjects
        self.vertices = vtx

    @property
    def vertices(self):
        return self._vertices

    @vertices.setter
    def vertices(self, vtx):
        self._vertices = vtx

class Boundary(CityModel):
    """CityJSON Geometry boundary"""
    def __init__(self, geom: List=list()):
        self.geometry = geom

    def get_geometry(self):
        """A nested array of vertex indices"""
        do sth with self.vertices # self.vertices comes from CityModel

However this is problematic in Python, because I want to have `__init__` methods for both CityModel and Boundary. While in the case above, `Boundary.__init__()` overrides `CityModel.__init__()`, making CityModel.vertices inaccessible (AttributeError) from the Boundary instances. Setting the vertices attribute outside `CityModel.__init__()` would require to instantiate a CityModel object in two steps, and would only work if vertices is not a property and there are `get_/set_vertices()` methods, which is not really pythonic. This is all because in Python everything is public. There are no real private attributes and methods as in C++.


In [ ]:
citymodel = CityModel(cityobjects)
citymodel.set_vertices(vertices)

Also, if Boundary is a subclass of CityModel, then Boundary gets all the methods, class attributes from CityModel. It just doesn't make sense to have a `get_cityobjects()` method in a Boundary instance... 

Finally, having a central vertex list would make it quite complicated to modify (esp. delete) boundaries, since on every delete, *all* the CityObjects and their boundaries would need to be traversed and the vertex indices updated.


### 2. Feature generation

How do we operate with 3D geometry? Do we cast to something from some library that has 3D geom? Or just provide getters for the vertices?

Python libraries with 3D geoms:

+ [open3d](http://www.open3d.org/docs/python_api/open3d.geometry.Geometry3D.html#open3d.geometry.Geometry3D)

#### Compute the volume of a building.

In [ ]:
def compute_volume(geometry):
    if geometry.boundaries is empty:
        return 0
    if geometry.type == 'Solid':
        if geometry.lod < 2:
            figure out what surface is what
        else:
            use the surface semantics
        compute the volume
    elif geometry.type == 'Point':
        raise TypeError("Cannot compute the volume of Point geometry")
    return volume

cm = cjio.load("some_model.json")
for building in cm.get_cityobjects("building"):
    # we need to check if the parent has geometry, but also if the child has geometry, because it is not prescribed how this should be
    volume_parent = compute_volume(building.geometry)
    
    for child in building.get_children():
    # actually, we need to do this recursively in order to visit the children of children too, because it is
    # not defined how many level deep we need to go
        geometry = child.get_geometry()
        volume_child = compute_volume(geometry)
        
    volume = volume_parent + volume_child

Get shape descriptors from the footrpints

Compute roof overhang as a distance between footprint and roofprint

Compute roof levels and roof types

(compare model to point cloud)

**!!! the most important software feature here is to allow the users to easily integrate their own cityobject/geometry processing functions with cjio !!!**

### 3. Export

Save cityobject attributes in tabular format (eg. tsv)

Save cityobject attributes in pandas dataframe

### 4. ML

Use the tabular output as input for scikit-learn (or any library). One can use `feather` to transport the objects to R.

## Alternative approaches

### Convert the CityJSON properties (dict keys in Python) into object attributes

The `json` library loads a json file into a Dictionary. Normally, working with a CityJSON file requires operating on this dictionary. It would be nicer if one can access the CityObjects of a CityModel as attributes of the CityModel. This is seemingly easily achieved with the [addict](https://github.com/mewwts/addict) library. 


In [1]:
import json
import addict

with open("./example_data/delft.json", 'r') as fin:
    raw_j = json.load(fin)

parsed = addict.Dict(raw_j)
v = parsed.vertices
co = parsed.CityObjects

However UUIDs break things a bit.

In [5]:
[print(uuid) for uuid in list(co.keys())[:2]]

b0a8da4cc-2d2a-11e6-9a38-393caa90be70
b1105d28c-00ba-11e6-b420-2bdcc4ab5d7f


[None, None]

In [6]:
co.b0a8da4cc-2d2a-11e6-9a38-393caa90be70

SyntaxError: invalid syntax (<ipython-input-6-bde99c540ce4>, line 1)

In [7]:
co.'b0a8da4cc-2d2a-11e6-9a38-393caa90be70'

SyntaxError: invalid syntax (<ipython-input-7-902e447f4364>, line 1)

And we end up falling back to the regular way of using dictionaries, thus `addict` doesn't make much of a difference.

In [8]:
co['b0a8da4cc-2d2a-11e6-9a38-393caa90be70']


{'attributes': {'bgt_status': 'bestaand',
  'bronhouder': 'G0503',
  'class': 'dek',
  'creationdate': '2014-07-09',
  'eindregistratie': '',
  'hoortbijtypeoverbrugging': 'waardeOnbekend',
  'inonderzoek': '0',
  'lokaalid': 'G0503.032e68f09d7049cce0532ee22091b28c',
  'lv_publicatiedatum': '2016-06-07T16:22:15.000',
  'namespace': 'NL.IMGeo',
  'overbruggingisbeweegbaar': '0',
  'plus_status': 'geenWaarde',
  'relatievehoogteligging': '1',
  'terminationdate': '',
  'tijdstipregistratie': '2016-05-17T13:43:18.000'},
 'geometry': [{'boundaries': [[[18260, 3129, 3131]],
    [[18260, 3165, 3129]],
    [[3136, 3137, 3135]],
    [[3136, 3138, 3137]],
    [[3166, 3165, 18260]],
    [[3136, 3163, 3138]],
    [[3138, 3140, 3139]],
    [[3139, 18261, 3144]],
    [[3144, 3147, 3146]],
    [[3144, 18261, 3147]],
    [[3139, 3140, 18261]],
    [[3138, 3163, 3140]],
    [[3164, 3136, 3165]],
    [[3163, 3164, 3168]],
    [[3163, 3136, 3164]],
    [[3128, 3166, 18260]],
    [[3164, 3165, 3166]],
  